In [8]:
import pandas as pd
import numpy as np
import time
import seaborn as sns
from datetime import datetime, timedelta 
import sqlite3
pd.set_option("display.max_columns", None)
import warnings
warnings.filterwarnings("ignore")

In [9]:
#mes = diciembre
#cierre mes = '20221231'
def Tabla_e(mes,cierre_mes):   
    #Lectura Datos
    tabla_e = pd.read_csv(r"..\Claro\Stocks mensuales\Stock HFC {}.csv".format(mes))
    tabla_e = tabla_e.loc[(tabla_e.ESTADO_SERVICIO == 'Activo')&(tabla_e.ESTADO_CLIENTE == 'Cliente')]
    dic = pd.read_excel(r"Datos-Tabla E CLARO\DICCIONARIO_V2.xlsb",sheet_name = 'TODO PLAN HOGAR', engine='pyxlsb')

    #correcion codigo plan actual
    y = tabla_e.loc[tabla_e.PLAN_NEW.isnull() == True]
    y.PLAN_NEW = y.PLAN_CONTRATADO.values
    y.CODIGO_PLAN_NEW = y.CODIGO_PLAN.values
    x =  tabla_e.loc[tabla_e.PLAN_NEW.isnull() != True]
    tabla_e = pd.concat([y,x]).sort_index()

    #procesamiento datos
    tabla_e= tabla_e.drop(columns=['PLAN_CONTRATADO', 'CODIGO_PLAN','FECHA_INSTALACION_SERVICIO'])
    tabla_e = tabla_e.merge(dic[['NOMBRE PLAN','PLAN ID','MONTO PLAN FINAL', 'PLAY','PRECIO_TV $', 'PRECIO_TLF $', 'PRECIOIBA $', 'MONTO MULTISERVICIO',
           'DSCTO_MULTISERVICIO_TV -$', 'DSCTO_MULTISERVICIO_TLF -$','DSCTO_MULTISERVICIO_IBA -$', 'DESCUENTO MULTISERVICIO','CLAVE_TV',
           'CLAVE_TLF', 'CLAVE_IBA', 'NOMBRE_TV', 'NOMBRE_TLF', 'NOMBRE_IBA','VELOCIDAD INTERNET','TIPO TECNOLOGÍA']], how = 'left', left_on = 'CODIGO_PLAN_NEW',right_on='PLAN ID')


    #creacion columnas de descuentos
    adq = pd.read_csv(r"..\Claro\INPUTS\Adquisicion\Adquisicion Activos\{} Descuentos Adquisicion Activos.csv".format(cierre_mes), delimiter = ';')
    ret = pd.read_csv(r"..\Claro\INPUTS\Retencion\Retencion Activos\{} Descuentos Retencion Activos.csv".format(cierre_mes), delimiter = ';')

    #renombrar columnas para que quede similar a la data de VTR
    tabla_e.rename(columns = {'PLAN_NEW':'NOMBRE_PROMOCION','CODIGO_PLAN_NEW':'ID_PROMO','PLAY':'N_EMPAQUETAMIENTO',
                             'PRECIO_TV $':'RENTA_CABLE','PRECIO_TLF $':'RENTA_FONO','PRECIOIBA $':'RENTA_INTERNET','DESCUENTO MULTISERVICIO':'DCTO_MULTISERVICIO',
                             'CLAVE_TV':'ID_CABLE','CLAVE_TLF':'ID_FONO','CLAVE_IBA':'ID_INTERNET'},inplace = True)

    adq1 = adq.groupby('ID_CLIENTE').agg(VALOR_DESC_ADQUISICION = ('MONTO_DSCTO','sum')).reset_index()
    adq1.rename(columns = {'ID_CLIENTE' : 'RUT'}, inplace = True)

    ret1 = ret.groupby('ID_CLIENTE').agg(VALOR_DESC_RETENCION = ('MONTO_DSCTO','sum')).reset_index()
    ret1.rename(columns = {'ID_CLIENTE' : 'RUT'}, inplace = True)


    # merge de columnas de descuento

    tabla_e = tabla_e.merge(adq1,how = 'left', on = ['RUT'])
    tabla_e = tabla_e.merge(ret1,how = 'left', on = ['RUT'])

    tabla_e['VALOR_DESC_ADQUISICION'] = tabla_e['VALOR_DESC_ADQUISICION'].fillna(0)
    tabla_e['VALOR_DESC_RETENCION'] = tabla_e['VALOR_DESC_RETENCION'].fillna(0)
    tabla_e['RENTA_PRINCIPAL'] = tabla_e['MONTO PLAN FINAL']
    tabla_e['RENTA_PRINCIPAL_2'] = tabla_e['RENTA_PRINCIPAL'] + tabla_e['VALOR_DESC_ADQUISICION'] + tabla_e['VALOR_DESC_RETENCION']
    tabla_e['N_RGU'] = np.where(tabla_e.N_EMPAQUETAMIENTO == '1 PLAY',1,
                            np.where(tabla_e.N_EMPAQUETAMIENTO == '2 PLAY',2,3))

    conditions_bundle = [((~tabla_e['ID_CABLE'].isnull()) & (tabla_e['ID_INTERNET'].isnull()) & (tabla_e['ID_FONO'].isnull())),
                       ((~tabla_e['ID_CABLE'].isnull()) & (tabla_e['ID_INTERNET'].isnull()) & (~tabla_e['ID_FONO'].isnull())),
                       ((~tabla_e['ID_CABLE'].isnull()) & (~tabla_e['ID_INTERNET'].isnull()) & (~tabla_e['ID_FONO'].isnull())),
                       ((~tabla_e['ID_CABLE'].isnull()) & (~tabla_e['ID_INTERNET'].isnull()) & (tabla_e['ID_FONO'].isnull())),
                       ((tabla_e['ID_CABLE'].isnull()) & (tabla_e['ID_INTERNET'].isnull()) & (~tabla_e['ID_FONO'].isnull())),
                       ((tabla_e['ID_CABLE'].isnull()) & (~tabla_e['ID_INTERNET'].isnull()) & (~tabla_e['ID_FONO'].isnull())),
                       ((tabla_e['ID_CABLE'].isnull()) & (~tabla_e['ID_INTERNET'].isnull()) & (tabla_e['ID_FONO'].isnull()))]
    choices_bundle = ['C', 'C + F', '3PLAY', 'C + I', 'F', 'I + F', 'I']
    choices_bundle2 = [1, 2, 3, 2, 1, 2, 1]
    tabla_e['BUNDLE'] = np.select(conditions_bundle, choices_bundle, default='-1')
    tabla_e['NUM_RGU'] = np.select(conditions_bundle, choices_bundle2, default= 0)

    tabla_e['DSCTO_MULTISERVICIO_TV -$'] = tabla_e['DSCTO_MULTISERVICIO_TV -$'].fillna(0)
    tabla_e['DSCTO_MULTISERVICIO_TLF -$'] = tabla_e['DSCTO_MULTISERVICIO_TLF -$'].fillna(0)
    tabla_e['DSCTO_MULTISERVICIO_IBA -$'] = tabla_e['DSCTO_MULTISERVICIO_IBA -$'].fillna(0)
    tabla_e['DCTO_MULTISERVICIO'] = tabla_e['DCTO_MULTISERVICIO'].fillna(0)

    tabla_e = tabla_e.loc[(tabla_e['NOMBRE PLAN'].isnull() == False)&(tabla_e.TIPO_CLIENTE == 'PERSONA')]
    #transformacion dtypes
    tabla_e = tabla_e.astype({"MONTO PLAN FINAL":"int","RENTA_CABLE":"int","RENTA_FONO":"int","RENTA_INTERNET":"int","MONTO MULTISERVICIO":"int","DSCTO_MULTISERVICIO_TV -$":"int",
                              "DSCTO_MULTISERVICIO_TLF -$":"int","DSCTO_MULTISERVICIO_IBA -$":"int","DCTO_MULTISERVICIO":"int","VALOR_DESC_ADQUISICION":"int",
                              "VALOR_DESC_RETENCION":"int","RENTA_PRINCIPAL":"int","RENTA_PRINCIPAL_2":"int",})
    tabla_e['RENTA_FONO1'] = tabla_e['RENTA_FONO'] - tabla_e['DSCTO_MULTISERVICIO_TLF -$']
    tabla_e['RENTA_CABLE1'] = tabla_e['RENTA_CABLE'] - tabla_e['DSCTO_MULTISERVICIO_TV -$']
    tabla_e['RENTA_INTERNET1'] = tabla_e['RENTA_INTERNET'] - tabla_e['DSCTO_MULTISERVICIO_IBA -$']
    return tabla_e

In [10]:
tabla_e = Tabla_e('abril 2023','20230430')

In [11]:
tabla_e.to_csv(r"..\Claro\Tabla E Claro\202305_TablaE_Claro.csv",index = False)